# Statistiche di valutazione del modello addestrato

In [1]:
from medicane_utils.final_stats import evaluate_binary_classifier

In [45]:
import os
import torch
import numpy as np
import pandas as pd
from arguments import prepare_finetuning_args, Args
from dataset import build_dataset
from torch.utils.data import DataLoader
import models
from timm.models import create_model

from model_analysis import predict_label, get_path_pred_label, create_df_predictions, get_only_labels
from build_dataset import create_df_video_from_master_df, create_final_df_csv

output_dir = "../airmassRGB/supervised/" 

In [24]:
# devo ricreare un dataset perché serve un nuovo master df
df_data = pd.read_csv("master_data_2020_wID.csv", parse_dates=['datetime', 'start_time', 'end_time', 'delta_time'])
df_data['delta_time'] = pd.to_timedelta(df_data['delta_time'])


/tmp/ipykernel_3570705/3092187031.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data = pd.read_csv("master_data_2020_wID.csv", parse_dates=['datetime', 'start_time', 'end_time', 'delta_time'])


In [25]:
df_data

,path,datetime,tile_offset_x,tile_offset_y,label,x_pix,y_pix,name,source,id_cyc_unico,start_time,end_time,delta_time
0,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,0,0,0,[],[],NaN,[],0,NaT,NaT,NaT
1,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,213,0,0,[],[],NaN,[],0,NaT,NaT,NaT
2,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,426,0,0,[],[],NaN,[],0,NaT,NaT,NaT
3,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,639,0,0,[],[],NaN,[],0,NaT,NaT,NaT
4,../fromgcloud/airmass_rgb_20200101_1200.png,2020-01-01 12:00:00,852,0,0,[],[],NaN,[],0,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
279367,../fromgcloud/airmass_rgb_20201228_0100.png,2020-12-28 01:00:00,213,196,0,[],[],NaN,[],0,NaT,NaT,NaT
279368,../fromgcloud/airmass_rgb_20201228_0100.png,2020-12-28 01:00:00,426,196,0,[],[],NaN,[],0,NaT,NaT,NaT
279369,../fromgcloud/airmass_rgb_20201228_0100.png,2020-12-28 01:00:00,639,196,0,[],[],NaN,[],0,NaT,NaT,NaT
279370,../fromgcloud/airmass_rgb_20201228_0100.png,2020-12-28 01:00:00,852,196,0,[],[],NaN,[],0,NaT,NaT,NaT


In [44]:
df_videos = create_df_video_from_master_df(df_data, idxs=range(1,9), output_dir=output_dir)

Num video CON cicloni: 85, Num video SENZA cicloni: 755	Totale video tiles: 840		Bilanciamento video...
 video senza cicloni tenuti: 85
Creazione delle folder per i 85 video...
Salvati 1360 file - Erano già presenti 0 file - File totali 1360
Creazione delle folder per i 85 video...
Salvati 1360 file - Erano già presenti 0 file - File totali 1360
170 video per il periodo da 2020-02-19 11:00:00 a 2020-02-19 12:15:00

Num video CON cicloni: 24, Num video SENZA cicloni: 444	Totale video tiles: 468		Bilanciamento video...
 video senza cicloni tenuti: 24
Creazione delle folder per i 24 video...
Salvati 384 file - Erano già presenti 0 file - File totali 384
Creazione delle folder per i 24 video...
Salvati 384 file - Erano già presenti 0 file - File totali 384
48 video per il periodo da 2020-03-12 20:00:00 a 2020-03-12 21:15:00

Num video CON cicloni: 21, Num video SENZA cicloni: 231	Totale video tiles: 252		Bilanciamento video...
 video senza cicloni tenuti: 21
Creazione delle folder per i 21

In [47]:
df_dataset_csv_test = create_final_df_csv(df_videos, output_dir)
df_dataset_csv_test.to_csv("./val_supervised_oggi3.csv", index=False)

In [52]:
df_dataset_csv_test_p =df_dataset_csv_test.copy()
df_dataset_csv_test_p.path = df_dataset_csv_test.path.str.split('/').str.get(-1)

In [48]:

args = prepare_finetuning_args()
##### prende da val_supervised.csv
dataset_val, _ = build_dataset(is_train=False, test_mode=False, args=args)  

data_loader_val = DataLoader(
    dataset_val,
    batch_size=args.batch_size,
    shuffle=True,         # Per estrarre sample casuali
    num_workers=args.num_workers,
    pin_memory=args.pin_mem,
    drop_last=False
)

get_prediction = False

all_paths, all_preds, all_labels = get_only_labels(data_loader_val)
print(len(all_paths), len(all_preds), len(all_labels))

Number of the class = 2
890 890 890


In [60]:
# voglio prendere le predizioni
get_prediction = True
args.test_mode = True

# istanzia l'oggetto del modello 
print(f"Creating model: {args.model} (nb_classes={args.nb_classes})")
model = create_model(
    args.model,
    num_classes=args.nb_classes,
    drop_rate=0.0,
    drop_path_rate=args.drop_path,
    #attn_drop_rate=0.0,
    drop_block_rate=None,
    **args.__dict__
)

model.to(args.device)
model.eval()

Creating model: vit_giant_patch14_224 (nb_classes=2)
Caricamento del checkpoint da: ./output/checkpoint-best.pth
Caricato state_dict con chiave: model
Checkpoint caricato con successo!


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(3, 1408, kernel_size=(2, 14, 14), stride=(2, 14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
  

In [61]:
all_paths, all_preds, all_labels = get_path_pred_label(model, data_loader_val)

In [62]:
y_true = np.array(all_labels)
y_pred  = np.array(all_preds)
df_predictions = create_df_predictions(all_paths, all_preds, all_labels)
df_predictions.shape[0]

890

In [63]:
df_dataset_csv_test_p.merge(df_predictions, on='path')

,path,start,end,label,predictions,labels
0,19-02-2020_1215_639_0,1,16,1,0,1
1,19-02-2020_1335_639_0,1,16,1,0,1
2,19-02-2020_1455_639_0,1,16,1,0,1
3,19-02-2020_1615_639_0,1,16,1,1,1
4,19-02-2020_1735_639_0,1,16,1,1,1
...,...,...,...,...,...,...
885,21-04-2020_2335_639_0,1,16,0,1,0
886,20-04-2020_1015_0_0,1,16,0,0,0
887,22-04-2020_1255_0_196,1,16,0,0,0
888,22-04-2020_0335_1065_0,1,16,0,1,0


# Modifica delle label dal Master df per verifica del periodo di formazione e fine del ciclone


In [54]:
from dataset_labeling_study import aggiorna_label_distanza_temporale
from build_dataset import get_gruppi_date, create_tile_videos, group_df_by_offsets
from dataset_labeling_study import create_tile_videos_relabeling, create_df_video_from_master_df_relabeling

In [8]:
df_data = pd.read_csv("master_data_2020_wID.csv", parse_dates=['datetime', 'start_time', 'end_time', 'delta_time'])
#df_data.info()
df_data['delta_time'] = pd.to_timedelta(df_data['delta_time'])
df_data.shape[0]

/tmp/ipykernel_3570705/748525876.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data = pd.read_csv("master_data_2020_wID.csv", parse_dates=['datetime', 'start_time', 'end_time', 'delta_time'])


279372

In [18]:
nome_file = "all_data_full_tiles.csv"
#nome_file = "all_data_wo_overlap_tiles.csv"
df_data = pd.read_csv(nome_file, dtype={
        "path": 'string',
        "tile_offset_x": 'int16',
        "tile_offset_y": 'int16',
        "label": 'int16',
        "lat": 'object',
        "lon": 'object',
        "x_pix": 'object',
        "y_pix": 'object',
        "name": 'string',
        "source": 'string'
    }, parse_dates=['datetime'])
df_data.drop(columns="Unnamed: 0", inplace=True)
df_data.shape[0]

1413168

In [55]:
step = pd.Timedelta(minutes=40)
for i in range(1, 10):
    t = i * step
    print(t)
    aggiorna_label_distanza_temporale(df_data, soglia=t)

0 days 00:40:00
Colonna label_00_40 aggiunta
0 days 01:20:00
Colonna label_01_20 aggiunta
0 days 02:00:00
Colonna label_02_00 aggiunta
0 days 02:40:00
Colonna label_02_40 aggiunta
0 days 03:20:00
Colonna label_03_20 aggiunta
0 days 04:00:00
Colonna label_04_00 aggiunta
0 days 04:40:00
Colonna label_04_40 aggiunta
0 days 05:20:00
Colonna label_05_20 aggiunta
0 days 06:00:00
Colonna label_06_00 aggiunta


In [14]:
tile_mask = (df_data[['tile_offset_x','tile_offset_y']] == (852, 196)).all(axis=1)

In [56]:
#[['label', 'datetime', 'delta_time', 'label_00_20', 'label_03_00']]
# Seleziona tutte le colonne il cui nome inizia con 'label'
label_cols = [col for col in df_data.columns if col.startswith('label')]

# Crea un nuovo DataFrame solo con quelle colonne
#df_label = df_data.loc[tile_mask, label_cols+ ['path', 'tile_offset_x','tile_offset_y', 'datetime']]
df_label = df_data[label_cols+ ['path', 'tile_offset_x','tile_offset_y', 'datetime']]
df_label


,label,label_00_40,label_01_20,label_02_00,label_02_40,label_03_20,label_04_00,label_04_40,label_05_20,label_06_00,path,tile_offset_x,tile_offset_y,datetime
0,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,0,0,2020-01-01 12:00:00
11,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,1065,196,2020-01-01 12:00:00
10,1,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,852,196,2020-01-01 12:00:00
9,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,639,196,2020-01-01 12:00:00
7,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,213,196,2020-01-01 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279361,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20201228_0100.png,213,0,2020-12-28 01:00:00
279360,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20201228_0100.png,0,0,2020-12-28 01:00:00
279370,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20201228_0100.png,852,196,2020-12-28 01:00:00
279364,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20201228_0100.png,852,0,2020-12-28 01:00:00


In [57]:
df_videos_many_labels = create_df_video_from_master_df_relabeling(df_label)

Righe del df_video: 1044
Righe del df_video: 840
Righe del df_video: 468
Righe del df_video: 252
Righe del df_video: 468
Righe del df_video: 792
Righe del df_video: 708
Righe del df_video: 504
Righe del df_video: 708
Righe del df_video: 120
Righe del df_video: 1128
Righe del df_video: 48
Righe del df_video: 756
Righe del df_video: 864
Righe del df_video: 144
Righe del df_video: 60
Righe del df_video: 1464
Righe del df_video: 420
Righe del df_video: 612
Righe del df_video: 852
Righe del df_video: 1056
Righe del df_video: 1512
Righe del df_video: 648
Righe del df_video: 1080
Righe del df_video: 792


In [64]:
df_result = df_videos_many_labels.merge(df_predictions, on='path')
#df_videos.merge(val_dataset, on='path')

In [66]:
df_result

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths,label_00_40,label_01_20,label_02_00,label_02_40,label_03_20,label_04_00,label_04_40,label_05_20,label_06_00,predictions,labels
0,0,0,0,19-02-2020_1215_0_0,0,2020-02-19 11:00:00,2020-02-19 12:15:00,"[../fromgcloud/airmass_rgb_20200219_1100.png, ...",0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,19-02-2020_1455_0_0,0,2020-02-19 13:40:00,2020-02-19 14:55:00,"[../fromgcloud/airmass_rgb_20200219_1340.png, ...",0,0,0,0,0,0,0,0,0,0,0
2,12,0,0,20-02-2020_0415_0_0,0,2020-02-20 03:00:00,2020-02-20 04:15:00,"[../fromgcloud/airmass_rgb_20200220_0300.png, ...",0,0,0,0,0,0,0,0,0,0,0
3,13,0,0,20-02-2020_0535_0_0,0,2020-02-20 04:20:00,2020-02-20 05:35:00,"[../fromgcloud/airmass_rgb_20200220_0420.png, ...",0,0,0,0,0,0,0,0,0,0,0
4,17,0,0,20-02-2020_1055_0_0,0,2020-02-20 09:40:00,2020-02-20 10:55:00,"[../fromgcloud/airmass_rgb_20200220_0940.png, ...",0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,654,1065,196,20-04-2020_0055_1065_196,0,2020-04-19 23:40:00,2020-04-20 00:55:00,"[../fromgcloud/airmass_rgb_20200419_2340.png, ...",0,0,0,0,0,0,0,0,0,0,0
886,669,1065,196,20-04-2020_2055_1065_196,0,2020-04-20 19:40:00,2020-04-20 20:55:00,"[../fromgcloud/airmass_rgb_20200420_1940.png, ...",0,0,0,0,0,0,0,0,0,0,0
887,687,1065,196,21-04-2020_2055_1065_196,0,2020-04-21 19:40:00,2020-04-21 20:55:00,"[../fromgcloud/airmass_rgb_20200421_1940.png, ...",0,0,0,0,0,0,0,0,0,0,0
888,690,1065,196,22-04-2020_0055_1065_196,0,2020-04-21 23:40:00,2020-04-22 00:55:00,"[../fromgcloud/airmass_rgb_20200421_2340.png, ...",0,0,0,0,0,0,0,0,0,0,0


In [38]:
y_true = np.array(all_labels)
y_pred  = np.array(all_preds)
#np.save("y_true.npy",y_true)
#np.save("y_pred.npy",y_pred)


In [15]:
y_true = np.load("y_true.npy")
y_pred = np.load("y_pred.npy")

In [67]:
np.array(all_labels).shape # == y_true

(890,)

In [69]:
label_cols = [col for col in df_result.columns if col.startswith('label')]
y_preds = df_result[label_cols].values

In [70]:
all(df_result.label == y_preds[:,0])

True

In [37]:
y_preds.shape

(44, 12)

# Calcola le metriche

In [71]:
results = evaluate_binary_classifier(y_true, y_preds[:,0])
print("\n=== Metriche ===")
print(results)

=== Confusion Matrix ===
[[227 218]
 [218 227]]
True Positive: 227
True Negative: 227
False Positive: 218
False Negative: 218

=== Classification Report ===
              precision    recall  f1-score   support

         neg       0.51      0.51      0.51       445
         pos       0.51      0.51      0.51       445

    accuracy                           0.51       890
   macro avg       0.51      0.51      0.51       890
weighted avg       0.51      0.51      0.51       890


=== Metriche ===
accuracy               0.510112
balanced_accuracy      0.510112
precision              0.510112
recall                 0.510112
f1                     0.510112
matthews_corrcoef      0.020225
cohen_kappa            0.020225
true_negatives       227.000000
false_positives      218.000000
false_negatives      218.000000
true_positives       227.000000
Name: metrics, dtype: float64


In [39]:
results = evaluate_binary_classifier(y_true, y_pred)
print("\n=== Metriche ===")
print(results)

=== Confusion Matrix ===
[[238 113]
 [ 47 304]]
True Positive: 304
True Negative: 238
False Positive: 113
False Negative: 47

=== Classification Report ===
              precision    recall  f1-score   support

         neg       0.84      0.68      0.75       351
         pos       0.73      0.87      0.79       351

    accuracy                           0.77       702
   macro avg       0.78      0.77      0.77       702
weighted avg       0.78      0.77      0.77       702


=== Metriche ===
accuracy               0.772080
balanced_accuracy      0.772080
precision              0.729017
recall                 0.866097
f1                     0.791667
matthews_corrcoef      0.554042
cohen_kappa            0.544160
true_negatives       238.000000
false_positives      113.000000
false_negatives       47.000000
true_positives       304.000000
Name: metrics, dtype: float64
